# MapsTo in pySBOL2
The MapsTo class in the Synthetic Biology Open Language (SBOL) is used to explicitly state that two `ComponentInstance` objects, often from different levels of design hierarchy, represent the same biological entity. It is most often used when `ModuleDefinition` and `ComponentDefinition` objects are composed using `Module` and `ComponentInstance` objects.

`MapsTo` objects define how a `ComponentInstance` in a higher-level design relates to a `ComponentInstance` in a lower-level design through  and refinement relationships.

MapsTo objects have the following required properties:

* `local`: Refers to the `ComponentInstance` in the higher-level design.
* `remote`: Refers to the `ComponentInstance` in the lower-level design. The referenced instance must have access="public".
* `refinement`: Specifies how to interpret the relationship between the local and remote instances using a URI. For * `example`: http://sbols.org/v2#useRemote.

This example demonstrates linking a FunctionalComponent in a high-level toggle switch module to one in a lower-level LacI inverter using a MapsTo object. We will:

1. Define all necessary biological parts (`ComponentDefinition`): two proteins (LacI, cI) and two promoters (pLac, pR).
2. Define two `ModuleDefinition` objects for genetic inverters: one repressed by LacI and one repressed by cI.
3. Define a higher-level `ModuleDefinition` for the toggle switch itself.
4. Instantiate both inverters inside the toggle switch module.
5. Use `MapsTo` to connect the inputs and outputs of the inverters, forming the complete, mutually-repressive feedback loop.

For more information on the MapsTo class and its properties, refer to page 30 in the SBOL 2.x.x specification document.

In [1]:
import sbol2

In [2]:
doc = sbol2.Document()

# Set the namespace for all SBOL objects
sbol2.setHomespace('https://github.com/SynBioDex/SBOL-Notebooks')

## Define Shared Biological Parts

Here we define four `ComponentDefinition` objects representing the core parts of our toggle switch: the coding sequences for the two repressor proteins (LacI and cI) and the two promoters they regulate (pLac and pR). In a real design, these would be detailed with sequence information.

In [3]:
# Coding Sequences (CDS)
lacI_cds = sbol2.ComponentDefinition('LacI_CDS', sbol2.BIOPAX_DNA)
lacI_cds.name = 'LacI Coding Sequence'
lacI_cds.addRole(sbol2.SO_CDS)
doc.addComponentDefinition(lacI_cds)

cI_cds = sbol2.ComponentDefinition('cI_CDS', sbol2.BIOPAX_DNA)
cI_cds.name = 'Lambda cI Coding Sequence'
cI_cds.addRole(sbol2.SO_CDS)
doc.addComponentDefinition(cI_cds)

# Define Promoters
pLac = sbol2.ComponentDefinition('pLac', sbol2.BIOPAX_DNA)
pLac.name = 'pLac Promoter'
pLac.addRole(sbol2.SO_PROMOTER)
doc.addComponentDefinition(pLac)

pR = sbol2.ComponentDefinition('pR', sbol2.BIOPAX_DNA)
pR.name = 'pR Promoter'
pR.addRole(sbol2.SO_PROMOTER)
doc.addComponentDefinition(pR)

## Define the Inverter Modules (Lower-Level Subsystems)

A toggle switch is composed of two inverters. We define each as a `ModuleDefinition`. Each inverter has an input (the repressor protein), a regulated promoter, and an output (the protein it produces). We also add `Interaction` objects to specify the biological function: repression.

- **LacI Inverter**: Takes LacI as input to repress the pLac promoter, which produces the cI protein.
- **cI Inverter**: Takes cI as input to repress the pR promoter, which produces the LacI protein.

In [4]:
# 1. LacI Inverter Module (LacI represses production of cI)
lacI_inverter_md = sbol2.ModuleDefinition('LacI_Inverter')

# Functional Components of the LacI inverter
lacI_tf_fc = lacI_inverter_md.functionalComponents.create('laci_transcription_factor')
lacI_tf_fc.definition = lacI_cds
lacI_tf_fc.access = sbol2.SBOL_ACCESS_PUBLIC
lacI_tf_fc.direction = sbol2.SBOL_DIRECTION_IN

pLac_fc = lacI_inverter_md.functionalComponents.create('pLac_promoter')
pLac_fc.definition = pLac
pLac.access = sbol2.SBOL_ACCESS_PRIVATE

cI_gene_product_fc = lacI_inverter_md.functionalComponents.create('cI_gene_product')
cI_gene_product_fc.definition = cI_cds
cI_gene_product_fc.access = sbol2.SBOL_ACCESS_PUBLIC
cI_gene_product_fc.direction = sbol2.SBOL_DIRECTION_OUT



# 2. cI Inverter Module (cI represses production of LacI)
cI_inverter_md = sbol2.ModuleDefinition('cI_Inverter')

# Functional Components of the cI inverter
cI_tf_fc = cI_inverter_md.functionalComponents.create('cI_transcription_factor')
cI_tf_fc.definition = cI_cds
cI_tf_fc.access = sbol2.SBOL_ACCESS_PUBLIC
cI_tf_fc.direction = sbol2.SBOL_DIRECTION_IN

pR_fc = cI_inverter_md.functionalComponents.create('pR_promoter')
pR_fc.definition = pR
pR_fc.access = sbol2.SBOL_ACCESS_PRIVATE

lacI_gene_product_fc = cI_inverter_md.functionalComponents.create('LacI_gene_product')
lacI_gene_product_fc.definition = lacI_cds
lacI_gene_product_fc.access = sbol2.SBOL_ACCESS_PUBLIC
lacI_gene_product_fc.direction = sbol2.SBOL_DIRECTION_OUT

## Define the Toggle Switch Module (Higher-Level Circuit)

This `ModuleDefinition` represents the complete genetic toggle switch. From this high-level perspective, we only care about the two key players: the LacI protein and the cI protein.

In [5]:
toggle_md = sbol2.ModuleDefinition('toggle_switch')

# The toggle switch's behavior is defined by two protein pools
lacI_prot_fc = toggle_md.functionalComponents.create('LacI_protein')
lacI_prot_fc.definition = lacI_cds
lacI_prot_fc.direction  = sbol2.SBOL_DIRECTION_IN_OUT

cI_prot_fc = toggle_md.functionalComponents.create('cI_protein')
cI_prot_fc.definition = cI_cds
cI_prot_fc.direction  = sbol2.SBOL_DIRECTION_IN_OUT

## Add Inverter Modules to Toggle Switch (Composition)

We now instantiate both inverter modules within the toggle switch module. This reflects **module composition** in SBOL—building a larger system from smaller, defined subsystems.

In [6]:
# Instantiate the LacI inverter
laci_inverter_instance = toggle_md.modules.create('laci_inverter_inst')
laci_inverter_instance.definition = lacI_inverter_md

# Instantiate the cI inverter
ci_inverter_instance = toggle_md.modules.create('ci_inverter_inst')
ci_inverter_instance.definition = cI_inverter_md

In [7]:
# Interaction 1: LacI protein represses the pLac promoter (inhibiting cI production)
interaction1 = sbol2.Interaction('LacI_represses_pLac')
interaction1.interaction_type = sbol2.SBO_INHIBITION
toggle_md.interactions.add(interaction1)

# Creating participation objects
# LacI protein is the INHIBITOR
inhibitor1 = interaction1.participations.create('part_LacI_protein')
inhibitor1.roles = [sbol2.SBO_INHIBITOR]
inhibitor1.participant = lacI_prot_fc

# pLac promoter is what is being INHIBITED
inhibited1 = interaction1.participations.create('part_pLac_promoter')
inhibited1.roles = [sbol2.SBO_INHIBITED]
inhibited1.participant = pLac_fc


# Interaction 2: cI protein represses the pLambda promoter (inhibiting LacI production)
interaction2 = sbol2.Interaction('cI_represses_pLambda')
interaction2.interaction_type = sbol2.SBO_INHIBITION
toggle_md.interactions.add(interaction2)

# Participation 2.1: cI protein is the INHIBITOR
inhibitor2 = interaction2.participations.create('part_cI_protein')
inhibitor2.roles = [sbol2.SBO_INHIBITOR]
inhibitor2.participant = cI_prot_fc

# Participation 2.2: The cI-repressible promoter is what is being INHIBITED
inhibited2 = interaction2.participations.create('part_pR_promoter')
inhibited2.roles = [sbol2.SBO_INHIBITED]
inhibited2.participant = pR_fc

## Wire the Circuit with `MapsTo`

This is the key step. We use `MapsTo` objects to connect the components and form the feedback loop. We need four mappings to declare that:
1. The `LacI_protein` in the toggle switch is the same molecule that acts as the input to the LacI inverter.
2. The `cI_protein` in the toggle switch is the same molecule that is produced as the output of the LacI inverter.
3. The `cI_protein` in the toggle switch is the same molecule that acts as the input to the cI inverter.
4. The `LacI_protein` in the toggle switch is the same molecule that is produced as the output of the cI inverter.

This wiring correctly models the mutual repression: LacI represses cI production, and cI represses LacI production.

In [8]:
# Map 1: LacI protein is the input to the LacI inverter
map1 = laci_inverter_instance.mapsTos.create('map_laci_in')
# map1.refinement = sbol2.SBOL_REFINEMENT_USE_REMOTE
map1.local = lacI_tf_fc
map1.remote = lacI_prot_fc

# Map 2: cI protein is the output of the LacI inverter
map2 = laci_inverter_instance.mapsTos.create('map_ci_out')
# map2.refinement = sbol2.SBOL_REFINEMENT_USE_REMOTE
map2.local = cI_tf_fc
map2.remote = cI_prot_fc

# Map 3: cI protein is the input to the cI inverter
map3 = ci_inverter_instance.mapsTos.create('map_ci_in')
# map3.refinement = sbol2.SBOL_REFINEMENT_USE_LOCAL
map3.local = cI_tf_fc
map3.remote = cI_prot_fc

# Map 4: LacI protein is the output of the cI inverter
map4 = ci_inverter_instance.mapsTos.create('map_laci_out')
# map3.refinement = sbol2.SBOL_REFINEMENT_USE_LOCAL
map4.local = lacI_tf_fc
map4.remote = lacI_prot_fc

## Finalize and Validate

We add all high-level modules to the document and check that the complete design conforms to SBOL specifications. If valid, it is saved as `toggle_switch_complete.xml`.

In [9]:
doc.addModuleDefinition(lacI_inverter_md)
doc.addModuleDefinition(cI_inverter_md)
doc.addModuleDefinition(toggle_md)

# Validate the document
report = doc.validate()

if report == "Valid.":
    doc.write("example_mapsto.xml")
else:
    print(report)

Invalid. sbol-12003:  Strong Validation Error:  The FunctionalComponent referenced by the participant property of a Participation MUST be contained by the ModuleDefinition that contains the Interaction which contains the Participation.  Reference: SBOL Version 2.3.0 Section 7.9.4 on page 45 :  https://github.com/SynBioDex/SBOL-Notebooks/ModuleDefinition/toggle_switch/LacI_represses_pLac/part_pLac_promoter/1   Validation failed.
